In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import PIL
import cv2

from tensorflow import keras
from tensorflow.keras.models import Sequential
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.optimizers import *
from tensorflow.keras import *
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D,BatchNormalization,Input, Lambda
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
IMAGE_SIZE = [224, 224]

In [3]:
train_path = 'dataset/train'
validation_path = 'dataset/test'

In [4]:
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

58900480/58889256 [==============================] - 395s 7us/step


In [5]:
for layer in vgg.layers:
  layer.trainable = False

In [6]:
from glob import glob
folders = glob('C:\\Users\\HP\\Desktop\\CobraShot\\dataset\\train\\*')
print(len(folders))

13


In [7]:
x = Flatten()(vgg.output)

In [8]:
prediction = Dense(len(folders), activation='softmax')(x)

In [9]:
model = Model(inputs=vgg.input, outputs=prediction)

In [10]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [11]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [21]:
train_dataset= ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

validation_dataset= ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)



#test_set = test_dataset.flow_from_directory(validation_path,
                                            #target_size = (224, 224),
                                            #batch_size = 32,
                                            #class_mode = 'categorical')

In [23]:
test_dataset = ImageDataGenerator(rescale = 1./255)
validation = ImageDataGenerator(rescale = 1./255)

In [26]:
training_set = train_dataset.flow_from_directory(train_path,
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

validation_set = train_dataset.flow_from_directory(train_path, 
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 1779 images belonging to 13 classes.
Found 1779 images belonging to 13 classes.


In [29]:
training_set.class_indices

{'Banded Racer': 0,
 'Checkered Keelback': 1,
 'Common Krait': 2,
 'Common Rat Snake': 3,
 'Common Sand Boa': 4,
 'Common Trinket': 5,
 'Green Tree Vine': 6,
 'Indian Rock Python': 7,
 'King Cobra': 8,
 'Monocled Cobra': 9,
 "Russell's Viper": 10,
 'Saw-scaled Viper': 11,
 'Spectacled Cobra': 12}

In [32]:
training_set.classes

array([ 0,  0,  0, ..., 12, 12, 12])